# WorkGraph example to run Molecular Dynamics

## Aim

As an example, we start from a structure, run an md simulation, compute descriptors, and then use a filtering function to split the resulting structures into `train.xyz`, `test.xyz`, and `valid.xyz`.

Load the aiida profile, structure, model and code:

In [6]:
from aiida import load_profile
load_profile()

Profile<uuid='60b17659a9844c4bbd3bef8de0a8f417' name='presto'>

In [7]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [8]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read, iread

# structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
init_structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [9]:
from aiida.orm import load_code
janus_code = load_code("janus@localhost")

In [10]:
from aiida.orm import Str, Float, Bool, Int, Dict
inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "device": Str("cpu"),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "ensemble": Str("NVT"),
    "struct": init_structure,
    "md_kwargs": Dict(
        {
            "steps": 10,
            "traj-every": 2
        }
    )
}

In [11]:
from aiida.plugins import CalculationFactory

mdCalc = CalculationFactory("mlip.md")
descriptorsCalc = CalculationFactory("mlip.descriptors")

# Single Descriptor

In [12]:
from aiida_workgraph import WorkGraph
wg = WorkGraph("MD_workgraph")
md_calc = wg.add_task(
    mdCalc,
    name="md_calc",
    **inputs
)


In [13]:
# To find inputs/outputs of mdcalc uncomment following: 
# mdCalc.get_description()["spec"]["outputs"].keys()

In [14]:
descriptors_calc = wg.add_task(
    descriptorsCalc,
    name="descriptors_calc",
    struct=md_calc.outputs.final_structure,
)

defining outputnode


In [15]:
from sample_split import process_and_split_data

split_data = wg.add_task(
    process_and_split_data,
    name="split_data",
    struct=descriptors_calc.outputs.xyz_output
)


In [16]:
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'MD_work…

In [17]:
wg.tasks

NodeCollection(parent = "MD_workgraph", nodes = ["graph_inputs", "graph_outputs", "graph_ctx", "md_calc", "descriptors_calc", "split_data"])

In [ ]:
# wg.run()

# Loop Descriptor

In [22]:
mdCalc.get_description()["spec"]["outputs"].keys()

dict_keys(['_attrs', 'remote_folder', 'remote_stash', 'retrieved', 'std_output', 'log_output', 'results_dict', 'summary', 'stats_file', 'traj_file', 'traj_output', 'final_structure'])

In [40]:
trajectory = md_calc.outputs.traj_output
print(trajectory)
output= StructureData(ase=read(trajectory, index=1))
print(output)

SocketAny(name='traj_output', value=None)


GraphDeferredIllegalOperationError: Illegal operation on a future value (Socket): boolean evaluation
Socket: result of node 'op_eq'

Fix:
You used a future in a boolean context (if/while/assert/and/or).
Wrap logic in a nested @node.graph.

General guidance:
  • Wrap logic in a nested @node.graph.
  • Or use the WorkGraph If zone for branching on predicates.
  • Or for loops, use the While zone or Map zone.

In [26]:
print('outputs of mdCalc:', md_calc.outputs)

outputs of mdCalc: TaskSocketNamespace(name='outputs', sockets=['remote_folder', 'remote_stash', 'retrieved', 'std_output', 'log_output', 'results_dict', 'summary', 'stats_file', 'traj_file', 'traj_output', 'final_structure', '_outputs', '_wait'])


In [29]:
# read the trajectory data with iread
with md_calc.outputs.traj_output.open(mode="r") as traj_file:
    traj = list(iread(traj_file, format="extxyz"))  
print(f"Number of frames in trajectory: {len(traj)}")

AttributeError: 'SocketAny' object has no attribute 'open'

In [51]:
from aiida_workgraph import WorkGraph

with WorkGraph("MD_Simple") as wg:
    
    # MD simulation
    md_task = wg.add_task(
        mdCalc,
        name="md",
        **inputs
    )
    trajectory = md_task.outputs.traj_output
   
    # output=StructureData(ase=read(trajectory, index=":"))
    # for i in enumerate(output):

    for i in range(1):
        output=StructureData(ase=read(trajectory, index=i))
        # Descriptors on intermediate structures
        desc_task = wg.add_task(
            descriptorsCalc,
            name=f"descriptors_{i}",
            code=inputs['code'],
            model=inputs['model'],
            arch=inputs['arch'],
            device=inputs['device'],
            metadata=inputs['metadata'],
            calc_per_element=Bool(True),
            struct=md_task.outputs.final_structure,
            trajectory_data=output,
            
        )
    # # Descriptors on final structure
    
    # desc_task = wg.add_task(
    #     descriptorsCalc,
    #     name="descriptors",
    #     code=inputs['code'],
    #     model=inputs['model'],
    #     arch=inputs['arch'],
    #     device=inputs['device'],
    #     metadata=inputs['metadata'],
    #     calc_per_element=Bool(True),
    #     struct=md_task.outputs.final_structure,
    # )

wg.run()

GraphDeferredIllegalOperationError: Illegal operation on a future value (Socket): boolean evaluation
Socket: result of node 'op_eq'

Fix:
You used a future in a boolean context (if/while/assert/and/or).
Wrap logic in a nested @node.graph.

General guidance:
  • Wrap logic in a nested @node.graph.
  • Or use the WorkGraph If zone for branching on predicates.
  • Or for loops, use the While zone or Map zone.

In [52]:
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'MD_Simp…